In [1]:
import torch
import torchvision
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torchvision.models as models
from torch.utils.data import DataLoader
import torchvision.datasets as datasets
import torchvision.transforms as transforms

In [2]:
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

In [3]:
train_dataset = torchvision.datasets.CIFAR10(root='./data', train=True, transform=transform, download=True)
test_dataset = torchvision.datasets.CIFAR10(root='./data', train=False, transform=transform, download=True)

Files already downloaded and verified
Files already downloaded and verified


In [4]:
batch_size = 64
device = "cuda:0"

In [75]:
class ResNet50(nn.Module):
    def __init__(self, hidden_units, out_size):
        super(ResNet50, self).__init__()
        self.resnet50 = models.resnet50(pretrained=True)
        self.resnet50.fc_layer = nn.Linear(2048, hidden_units)
        self.resnet50.classifier = nn.Linear(hidden_units, out_size)

    def forward(self, x):
        features = self.resnet50.conv1(x)
        features = self.resnet50.bn1(features)
        features = self.resnet50.relu(features)
        features = self.resnet50.maxpool(features)
        features = self.resnet50.layer1(features)
        features = self.resnet50.layer2(features)
        features = self.resnet50.layer3(features)
        fmaps_b4 = self.resnet50.layer4(features)
        out = F.adaptive_avg_pool2d(fmaps_b4, (1, 1)).view(fmaps_b4.size(0), -1)
        out = self.resnet50.fc_layer(out)
        out = self.resnet50.classifier(out)
        return out


In [76]:
model1 = ResNet50(hidden_units=128, out_size=2).to(device)

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [77]:
class_indices = [0,1]
train_dataset_1 = torch.utils.data.Subset(train_dataset, [i for i in range(len(train_dataset)) if train_dataset.targets[i] in class_indices])
test_dataset_1 = torch.utils.data.Subset(train_dataset, [i for i in range(len(train_dataset)) if train_dataset.targets[i] in class_indices])
train_loader_1 = torch.utils.data.DataLoader(train_dataset_1, batch_size=batch_size, shuffle=True, num_workers=2)
test_loader_1 = torch.utils.data.DataLoader(test_dataset_1, batch_size=batch_size, shuffle=False, num_workers=2)

In [78]:
class_indices = [2,3]
train_dataset_2 = torch.utils.data.Subset(train_dataset, [i for i in range(len(train_dataset)) if train_dataset.targets[i] in class_indices])
test_dataset_2 = torch.utils.data.Subset(train_dataset, [i for i in range(len(train_dataset)) if train_dataset.targets[i] in class_indices])
train_loader_2 = torch.utils.data.DataLoader(train_dataset_2, batch_size=batch_size, shuffle=True, num_workers=2)
test_loader_2 = torch.utils.data.DataLoader(test_dataset_2, batch_size=batch_size, shuffle=False, num_workers=2)

In [79]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model1.parameters(), lr=0.001)
num_epochs = 10

In [80]:
for epoch in range(num_epochs):
    model1.train()
    running_loss = 0.0

    for inputs, labels in train_loader_1:
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()

        outputs = model1(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    epoch_loss = running_loss / len(train_loader_1)
    print(f"Epoch [{epoch + 1}/{num_epochs}] Loss: {epoch_loss:.4f}")

Epoch [1/10] Loss: 0.1905
Epoch [2/10] Loss: 0.0882
Epoch [3/10] Loss: 0.0444
Epoch [4/10] Loss: 0.0937
Epoch [5/10] Loss: 0.0941
Epoch [6/10] Loss: 0.2943
Epoch [7/10] Loss: 0.1345
Epoch [8/10] Loss: 0.0913
Epoch [9/10] Loss: 0.0715
Epoch [10/10] Loss: 0.0461


In [91]:
correct_predictions = 0
total_samples = 0

with torch.no_grad():
    for images, labels in test_loader_1:
        images, labels = images.to(device), labels.to(device)
        outputs = model1(images)
        _, predicted = torch.max(outputs, 1)
        total_samples += labels.size(0)
        correct_predictions += (predicted == labels).sum().item()

accuracy = 100 * correct_predictions / total_samples
print(f'Accuracy : {accuracy:.2f}%')

Accuracy : 99.41%


In [103]:
model2 = ResNet50(hidden_units=128, out_size=2).to(device)

In [104]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model2.parameters(), lr=0.001)
num_epochs = 25

In [108]:
model1.eval()
kl_criterion = nn.KLDivLoss(reduction='batchmean').to(device)
ce_criterion = nn.CrossEntropyLoss().to(device)
alpha = 0.5

for epoch in range(num_epochs):
    model2.train()

    running_loss = 0.0

    for inputs, labels in train_loader_2:
        inputs, labels = inputs.to(device), labels.to(device)
        labels = labels % 2
        optimizer.zero_grad()
        with torch.no_grad():
            teacher_outputs = model1(inputs)

        outputs = model2(inputs)

        kl_loss = kl_criterion(torch.log_softmax(outputs, dim=1), torch.softmax(teacher_outputs, dim=1))
        ce_loss = ce_criterion(outputs, labels)
        loss = alpha * ce_loss + (1-alpha) * kl_loss
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    epoch_loss = running_loss / len(train_loader_2)
    print(f"Epoch [{epoch + 1}/{num_epochs}] Loss: {epoch_loss:.4f}")

Epoch [1/15] Loss: 0.3544
Epoch [2/15] Loss: 0.3365
Epoch [3/15] Loss: 0.3252
Epoch [4/15] Loss: 0.3124
Epoch [5/15] Loss: 0.3002
Epoch [6/15] Loss: 0.2947
Epoch [7/15] Loss: 0.2917
Epoch [8/15] Loss: 0.2793
Epoch [9/15] Loss: 0.2763
Epoch [10/15] Loss: 0.2775
Epoch [11/15] Loss: 0.2687
Epoch [12/15] Loss: 0.2727
Epoch [13/15] Loss: 0.2742
Epoch [14/15] Loss: 0.2639
Epoch [15/15] Loss: 0.2642


In [109]:
correct_predictions = 0
total_samples = 0

with torch.no_grad():
    for images, labels in test_loader_2:
        images, labels = images.cuda(), labels.cuda()
        outputs = model2(images)
        _, predicted = torch.max(outputs, 1)
        total_samples += labels.size(0)
        correct_predictions += (predicted == labels % 2).sum().item()

accuracy = 100 * correct_predictions / total_samples
print(f'Accuracy : {accuracy:.2f}%')

Accuracy : 93.70%


In [110]:
correct_predictions = 0
total_samples = 0

with torch.no_grad():
    for images, labels in test_loader_1:
        images, labels = images.cuda(), labels.cuda()
        outputs = model2(images)
        _, predicted = torch.max(outputs, 1)
        total_samples += labels.size(0)
        correct_predictions += (predicted == labels).sum().item()

accuracy = 100 * correct_predictions / total_samples
print(f'Accuracy : {accuracy:.2f}%')

Accuracy : 85.42%
